# Seminar on Graphs for NLP: Vector representations

## Plan for today:

#### 0. What a taxonomy is. Taxonomy Enrichment task.
#### 1. Graph Neural networks: GCN and GAT
#### 2. GATv2
#### 3. GraphBERT: Only Attention is Needed for Learning Graph Representations
#### 4. OpenHGNN library

# 0. Taxonomy

A taxonomy is a hierarchical structure of units in terms if class inclusion such that superordinate units in the hierarchy include, or subsume, all items in subordinate units. Taxonomies are typically represented as having tree structures.

![](https://www.digital-mr.com/media/cache/51/6f/516f493d37a7b4895f678843b6383e48.png)


Taxonomies can be represented as graphs!

Let us download the most popular and well-known taxonomy called WordNet. You may also use the `from nltk.corpus import wordnet as wn`, but keep in mind that you can operate with earlier versions.

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!curl -L -o 'wordnet_n_is_directed_12_en_synsets.zip' 'https://drive.google.com/u/0/uc?id=1TvWsvz8UC0RPKHBx2GRi-iChVG4oTz-m&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  206M  100  206M    0     0  81.5M      0  0:00:02  0:00:02 --:--:--  114M


In [ ]:
!unzip wordnet_n_is_directed_12_en_synsets.zip

Archive:  wordnet_n_is_directed_12_en_synsets.zip
   creating: wordnet_n_is_directed_1_en_synsets/
  inflating: wordnet_n_is_directed_1_en_synsets/link  
   creating: wordnet_n_is_directed_1_en_synsets/.ipynb_checkpoints/
  inflating: wordnet_n_is_directed_1_en_synsets/.ipynb_checkpoints/link-checkpoint  
  inflating: wordnet_n_is_directed_1_en_synsets/node  


In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from gensim.models.poincare import PoincareModel
import numpy as np
import time
import os

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
wn.synset("dog.n.01").hyponyms()

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

In [ ]:
path = f"wordnet_n_is_directed_1_en_synsets/"

link_path = os.path.join(path, "link")
node_path = os.path.join(path, "node")

In [ ]:
id2synset = {}
fasttext_dict = {}

with open(node_path) as f:
    for line in f:
        line_split = line.split("\t")
        id2synset[line_split[0].strip()] = line_split[-1].strip()
        fasttext_dict[line_split[-1].strip()] = np.array([float(num) for num in line_split[1:-1]])

In [ ]:
link_pairs = set()
with open(link_path) as f:
    for line in f:
        line_split = line.split("\t")
        link_pairs.add((id2synset[line_split[0].strip()], id2synset[line_split[-1].strip()]))

# 4. Graph Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

In [ ]:
import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import train_test_split_edges
import torch_geometric.transforms as T
from torch_geometric.data import Data

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


## Data preparation

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

fasttext = KeyedVectors(vector_size=300)
fasttext.add_vectors(list(fasttext_dict.keys()), list(fasttext_dict.values()))

In [ ]:
import networkx as nx

In [ ]:
G = nx.DiGraph()

for pair in link_pairs:
    G.add_edge(*pair)

In [ ]:
for pair in link_pairs:
    print(pair)
    break

('criterion.n.02', 'design_criteria.n.01')


In [ ]:
def create_edge_list(G):
    starts = []
    ends = []
    for left, right in G.edges:
        if left in fasttext.key_to_index and right in fasttext.key_to_index:
            starts.append(fasttext.key_to_index[left])
            ends.append(fasttext.key_to_index[right])
    return torch.tensor([starts, ends], dtype=torch.long)

In [ ]:
index_to_key = dict(map(reversed, fasttext.key_to_index.items()))

In [ ]:
edge_index = create_edge_list(G)

In [ ]:
edge_index.shape

torch.Size([2, 72370])

In [ ]:
x = torch.tensor([fasttext[index_to_key[int(i)]] for i in index_to_key], dtype=torch.float)

<ipython-input-28-ceafeacad9cc>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x = torch.tensor([fasttext[index_to_key[int(i)]] for i in index_to_key], dtype=torch.float)


In [ ]:
x.shape

torch.Size([78748, 300])

In [ ]:
data = Data(x=x, edge_index=edge_index)
#data = train_test_split_edges(data)

In [ ]:
from torch_geometric.transforms import RandomLinkSplit

In [ ]:
transform = RandomLinkSplit(is_undirected=True, split_labels=True)
train_data, val_data, test_data = transform(data)

### GCN and GAT Encoder

The following code snippet describes the Encoder module with GCN or GAT networks.

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, mode="gcn"):
        super(Encoder, self).__init__()
        if mode == "gcn":
            self.conv1 = pyg_nn.GCNConv(in_channels, 2 * out_channels, cached=True)
            self.conv2 = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True)
        elif mode == 'gat':
            self.conv1 = pyg_nn.GATConv(in_channels, 2 * out_channels)
            self.conv2 = pyg_nn.GATConv(2 * out_channels, out_channels)
        else:
            raise Exception("Encoder mode is not recognized, try gcn/gat")

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    writer.add_scalar("loss", loss.item(), epoch)
    return loss.item()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [ ]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

channels = 64
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())

CUDA availability: True


## Variational Graph Auto-Encoders

https://arxiv.org/pdf/1611.07308.pdf

The pipeline is working as follows: first, we train a graph autoencoder with GCN or GAT under the hoot. During the evaluation phase, the latent representations of the autoencoder are actually the embeddings we are looking for.

In [ ]:
model = pyg_nn.GAE(Encoder(300, channels, 'gcn')).to(dev)
x, train_pos_edge_index = train_data.x.to(dev), train_data.pos_edge_label_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 401):
    loss = train(epoch)
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)
    writer.add_scalar("AUC", auc, epoch)
    writer.add_scalar("AP", ap, epoch)
    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}, Loss: {:.4f}'.format(epoch, auc, ap, loss))

Epoch: 010, AUC: 0.8323, AP: 0.8188, Loss: 0.9427
Epoch: 020, AUC: 0.8602, AP: 0.8515, Loss: 0.8738
Epoch: 030, AUC: 0.8825, AP: 0.8768, Loss: 0.8399
Epoch: 040, AUC: 0.8897, AP: 0.8882, Loss: 0.8183
Epoch: 050, AUC: 0.8939, AP: 0.8942, Loss: 0.8054
Epoch: 060, AUC: 0.8984, AP: 0.8996, Loss: 0.8016
Epoch: 070, AUC: 0.9007, AP: 0.9029, Loss: 0.7923
Epoch: 080, AUC: 0.9032, AP: 0.9061, Loss: 0.7850
Epoch: 090, AUC: 0.9072, AP: 0.9100, Loss: 0.7864
Epoch: 100, AUC: 0.9097, AP: 0.9124, Loss: 0.7830
Epoch: 110, AUC: 0.9108, AP: 0.9135, Loss: 0.7811
Epoch: 120, AUC: 0.9120, AP: 0.9150, Loss: 0.7746
Epoch: 130, AUC: 0.9132, AP: 0.9163, Loss: 0.7733
Epoch: 140, AUC: 0.9142, AP: 0.9173, Loss: 0.7732
Epoch: 150, AUC: 0.9154, AP: 0.9184, Loss: 0.7723
Epoch: 160, AUC: 0.9154, AP: 0.9189, Loss: 0.7705
Epoch: 170, AUC: 0.9168, AP: 0.9204, Loss: 0.7668
Epoch: 180, AUC: 0.9170, AP: 0.9208, Loss: 0.7693
Epoch: 190, AUC: 0.9163, AP: 0.9206, Loss: 0.7653
Epoch: 200, AUC: 0.9174, AP: 0.9216, Loss: 0.7621


#### Examples

Let us see the nearest neighbours for the unseen words from the test set.

In [ ]:
model.eval()
new_x = torch.tensor([fasttext[index_to_key[i]] for i in index_to_key], dtype=torch.float).to(dev)
z = model.encode(new_x, train_pos_edge_index)

In [ ]:
id2syns = {}
syns2id = {}
with open('wordnet_n_is_directed_1_en_synsets/node') as f:
    for line in f:
        id2syns[line.split()[0]] = line.split()[-1]
        syns2id[line.split()[-1]] = line.split()[0]

In [ ]:
par2orph = {}
orph2par = {}
with open('wordnet_n_is_directed_1_en_synsets/link') as f:
    for line in f:
        par_id = line.split()[0]
        child_id = line.split()[-1]

        if "ORPHAN_" in id2syns[child_id]:
            par2orph[id2syns[par_id]] = id2syns[child_id]
            orph2par[id2syns[child_id]] = id2syns[par_id]

In [ ]:
c = 0
for word in fasttext.key_to_index:
    if ".n." not in word:
        cur_index = fasttext.key_to_index[word]
        tensor_ = torch.tensor([[cur_index]*(len(G.nodes)), [i for i in range(0, len(G.nodes))]])
        results = model.decode(z, tensor_)
        top10 = list(reversed(sorted([(index_to_key[i], round(float(score.cpu().detach().float()), 4)) for i, score in enumerate(results)], key=lambda x: x[1])))[:10]
        print(orph2par[word], ":", top10)
        print("="*10)
        c += 1
        if c == 20:
            break

course.n.04 : [('action.n.06', 0.9923), ('action.n.09', 0.9923), ('act.n.05', 0.9918), ('pump_action.n.01', 0.9906), ('job_action.n.01', 0.9891), ('piano_action.n.01', 0.989), ('movement.n.10', 0.9875), ('police_action.n.01', 0.9871), ('gunlock.n.01', 0.9859), ('course.n.04', 0.9859)]
recovery.n.03 : [('rescue.n.01', 0.9973), ('ORPHAN_100000001', 0.9966), ('recovery.n.03', 0.992), ('purge.n.02', 0.9875), ('purification.n.01', 0.9859), ('sanitation.n.02', 0.9858), ('sterilization.n.02', 0.9858), ('due.n.02', 0.9856), ('scrub.n.02', 0.9856), ('housecleaning.n.02', 0.9853)]
disappearance.n.01 : [('three-d.n.01', 0.9825), ('due.n.02', 0.9377), ('ORPHAN_100000002', 0.919), ('inflow.n.01', 0.9162), ('ripple.n.02', 0.9108), ('special_effect.n.01', 0.9039), ('iris.n.02', 0.9015), ('financial_loss.n.01', 0.9011), ('bilocation.n.01', 0.8988), ('sound_effect.n.01', 0.8963)]
hit.n.03 : [('base_hit.n.01', 0.9774), ('act.n.05', 0.9618), ('blow.n.02', 0.9537), ('hit.n.01', 0.9475), ('ORPHAN_100000003

# Graph Attention Networks v2 (GATv2)

This is a PyTorch implementation of the GATv2 operator from the paper How Attentive are Graph Attention Networks?.

https://nn.labml.ai/graphs/gatv2/index.html

## GraphBERT

https://github.com/jwzhanggy/Graph-Bert

Yet another model for embedding generation is GraphBert. Instead of feeding large input graph, we train GRAPH-BERT with sampled subgraphs within their local contexts. The input vector embeddings to be fed to the graphtransformer model actually cover four parts: (1) raw feature vector embedding, (2) Weisfeiler-Lehman absolute role embedding, (3) intimacy based relative positional embedding, and (4) hop based relative distance embedding, respectively.

GRAPH-BERT is trained with the node attribute reconstruction and structure recovery tasks.

![](https://github.com/jwzhanggy/Graph-Bert/raw/master/result/screenshot/model.png)

## Subgraph Sampling

![](https://i.ibb.co/5cbjJZ6/photo-2021-12-07-16-41-32.jpg)

## Positional embeddings

### Weisfeiler-Lehman Absolute Role Embedding

![](https://i.ibb.co/bgT7gqb/wl.png)

### Intimacy based Relative Positional Embedding

![](https://i.ibb.co/34FvCf0/photo-2021-12-07-16-52-30.jpg)

### Hop based Relative Distance Embedding
![](https://i.ibb.co/tCzRcfK/hops-drawio.png)

Actually, you are simply expected to run two scripts: `script_1_preprocess.py` and `script_2_pre_train.py`

In [ ]:
!git clone https://github.com/jwzhanggy/Graph-Bert.git

Cloning into 'Graph-Bert'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 450 (delta 106), reused 79 (delta 78), pack-reused 314
Receiving objects: 100% (450/450), 2.23 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [ ]:
%cd Graph-Bert
!python3 script_1_preprocess.py

/content/Graph-Bert
************ Start ************
WL, dataset: cora
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 1
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 2
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 3
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 4
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 5
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 6
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 7
Loading cora dataset...
**********

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 script_2_pre_train.py

Traceback (most recent call last):
  File "/content/Graph-Bert/script_2_pre_train.py", line 5, in <module>
    from code.MethodBertComp import GraphBertConfig
  File "/content/Graph-Bert/code/MethodBertComp.py", line 11, in <module>
    from transformers.modeling_bert import BertPredictionHeadTransform, BertAttention, BertIntermediate, BertOutput
ModuleNotFoundError: No module named 'transformers.modeling_bert'


## View and evaluate results

In [ ]:
!gdown 1IAfd9tRgtVtdosM5vuDdxh-VSBFp3mzI
!gdown 1LItbxEcchOfU4TrlLBZjQweC8jpQ3b3Q
!gdown 1VLLLyu9YyLX3uCojiTm_VLtgK2gKCCfW
!gdown 1h5sSbFeCJbouH96fKIZDF2xugNiKf3La

Downloading...
From: https://drive.google.com/uc?id=1IAfd9tRgtVtdosM5vuDdxh-VSBFp3mzI
To: /content/Graph-Bert/MethodGraphBertGraphRecovery_model_test_embeddings.txt
100% 258k/258k [00:00<00:00, 133MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LItbxEcchOfU4TrlLBZjQweC8jpQ3b3Q
To: /content/Graph-Bert/MethodGraphBertGraphRecovery_model_train_embeddings_.txt
100% 196M/196M [00:05<00:00, 34.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VLLLyu9YyLX3uCojiTm_VLtgK2gKCCfW
To: /content/Graph-Bert/MethodGraphBertNodeConstruct_model_train_embeddings_.txt
100% 288M/288M [00:09<00:00, 31.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h5sSbFeCJbouH96fKIZDF2xugNiKf3La
To: /content/Graph-Bert/MethodGraphBertNodeConstruct_model_test_embeddings.txt
100% 362k/362k [00:00<00:00, 132MB/s]


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
graphBertNode_train = KeyedVectors.load_word2vec_format("MethodGraphBertNodeConstruct_model_train_embeddings_.txt")
graphBertNode_test = KeyedVectors.load_word2vec_format("MethodGraphBertNodeConstruct_model_test_embeddings.txt")

In [ ]:
graphBertNode_train.similar_by_word("dog.n.01")

[('hound.n.01', 0.8980603814125061),
 ('working_dog.n.01', 0.8848254680633545),
 ('dandy.n.01', 0.8751208782196045),
 ('old_man.n.01', 0.8639864325523376),
 ('professional.n.01', 0.8605043888092041),
 ('gravida.n.02', 0.849956750869751),
 ('child.n.02', 0.8499069809913635),
 ('spaniel.n.01', 0.8490362167358398),
 ('subordinate.n.01', 0.8471304178237915),
 ('parent.n.01', 0.8452426195144653)]

In [ ]:
fasttext.similar_by_word("dog.n.01")

[('dog.n.03', 0.9556826949119568),
 ('seizure-alert_dog.n.01', 0.9394533634185791),
 ('working_dog.n.01', 0.9265448451042175),
 ('dog_breeding.n.01', 0.9224575757980347),
 ('hunting_dog.n.01', 0.9179456233978271),
 ('dog_biscuit.n.01', 0.9155749678611755),
 ('ORPHAN_100000208', 0.9101989269256592),
 ('dog_catcher.n.01', 0.9075720906257629),
 ('raccoon_dog.n.01', 0.9067509770393372),
 ('dalmatian.n.02', 0.9053666591644287)]

In [ ]:
wn.synset("dog.n.01").hypernyms()

[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

# OpenHGNN library

https://github.com/BUPT-GAMMA/OpenHGNN

In [ ]:
pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 21.4 MB/s eta 0:00:00


In [ ]:
pip install openhgnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.7/230.7 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openhgnn: filename=openhgnn-0.4.0-py3-none-any.whl size=297048 sha256=b675b921fa182061a58b8c9c363a1a357babbd1018cbde4c6859148bb4d605f2
  Stored in directory: /root/.cache/pip/wheels/5e/23/6b/e05cfeeb232b3b82a75ccf9a49d9884602e7dc8dc375c85e84
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=b67e8168dc8f9e0e8c2a0f2b2ea886aaadc42f41b2a30e4d49cc6beeb6

In [ ]:
%cd ..

/


In [ ]:
!git clone https://github.com/BUPT-GAMMA/OpenHGNN

Cloning into 'OpenHGNN'...
remote: Enumerating objects: 6676, done.
remote: Counting objects: 100% (505/505), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 6676 (delta 322), reused 438 (delta 289), pack-reused 6171
Receiving objects: 100% (6676/6676), 15.26 MiB | 8.22 MiB/s, done.
Resolving deltas: 100% (4710/4710), done.


In [ ]:
# from networkx.algorithms.centrality import edge_betweenness_centrality

In [ ]:
!python ./OpenHGNN/main.py -m GTN -d imdb4GTN -t node_classification -g -1 --use_best_config

Traceback (most recent call last):
  File "//./OpenHGNN/main.py", line 8, in <module>
    from openhgnn.experiment import Experiment
  File "/OpenHGNN/openhgnn/__init__.py", line 4, in <module>
    from .trainerflow import *
  File "/OpenHGNN/openhgnn/trainerflow/__init__.py", line 89, in <module>
    from .slice_trainer import SLiCETrainer
  File "/OpenHGNN/openhgnn/trainerflow/slice_trainer.py", line 11, in <module>
    from networkx.algorithms.centrality.betweenness import edge_betweenness
ImportError: cannot import name 'edge_betweenness' from 'networkx.algorithms.centrality.betweenness' (/usr/local/lib/python3.10/dist-packages/networkx/algorithms/centrality/betweenness.py)
